<a href="https://colab.research.google.com/github/CTRLFreaks-ai/Lung_Prediction_Model/blob/main/Lung_Cancer_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ajavasi","key":"15d6733a37759320f82ff51c62428fb6"}'}

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d thedevastator/cancer-patients-and-air-pollution-a-new-link

rm: cannot remove '/root/.kaggle': No such file or directory
  0% 0.00/7.36k [00:00<?, ?B/s]
100% 7.36k/7.36k [00:00<00:00, 15.5MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('cancer-patients-and-air-pollution-a-new-link.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(f"using device: {device}")



2.1.0+cu121
using device: cuda


In [ ]:
data = pd.read_csv('cancer patient data sets.csv')
data = data.drop(columns=['index','Patient Id'])
data["Level"] = data["Level"].map({"Low":0, "Medium": 0.5, "High": 1})
data = data.rename(columns={"Level":"target"})
data["Age"] = data["Age"].div(100)
for col in data.columns:
  data[col] = data[col].astype('float32')
  if col != "Age" and col != "target":
    data[col] = data[col].div(10)
data

,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,Obesity,...,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring,target
0,0.33,0.1,0.2,0.4,0.5,0.4,0.3,0.2,0.2,0.4,...,0.3,0.4,0.2,0.2,0.3,0.1,0.2,0.3,0.4,0.0
1,0.17,0.1,0.3,0.1,0.5,0.3,0.4,0.2,0.2,0.2,...,0.1,0.3,0.7,0.8,0.6,0.2,0.1,0.7,0.2,0.5
2,0.35,0.1,0.4,0.5,0.6,0.5,0.5,0.4,0.6,0.7,...,0.8,0.7,0.9,0.2,0.1,0.4,0.6,0.7,0.2,1.0
3,0.37,0.1,0.7,0.7,0.7,0.7,0.6,0.7,0.7,0.7,...,0.4,0.2,0.3,0.1,0.4,0.5,0.6,0.7,0.5,1.0
4,0.46,0.1,0.6,0.8,0.7,0.7,0.7,0.6,0.7,0.7,...,0.3,0.2,0.4,0.1,0.4,0.2,0.4,0.2,0.3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.44,0.1,0.6,0.7,0.7,0.7,0.7,0.6,0.7,0.7,...,0.5,0.3,0.2,0.7,0.8,0.2,0.4,0.5,0.3,1.0
996,0.37,0.2,0.6,0.8,0.7,0.7,0.7,0.6,0.7,0.7,...,0.9,0.6,0.5,0.7,0.2,0.4,0.3,0.1,0.4,1.0
997,0.25,0.2,0.4,0.5,0.6,0.5,0.5,0.4,0.6,0.7,...,0.8,0.7,0.9,0.2,0.1,0.4,0.6,0.7,0.2,1.0
998,0.18,0.2,0.6,0.8,0.7,0.7,0.7,0.6,0.7,0.7,...,0.3,0.2,0.4,0.1,0.4,0.2,0.4,0.2,0.3,1.0


In [ ]:
class Lung_Cancer_Model(nn.Module):
  def __init__(self):
    super(Lung_Cancer_Model, self).__init__();
    input_size = 23
    hidden_size_1 = 10
    output_size = 1
    self.fc1 = nn.Linear(input_size, hidden_size_1)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size_1, output_size)
  def forward(self, x):
    x = x.view(x.size(0), -1)
    x = self.fc1(x)
    x = self.relu1(x)
    x = self.fc2(x)
    return x

In [ ]:
class LungDataset(Dataset):
  def __init__(self, dataframe, transform = None):
    self.data = dataframe
    self.transform = transform
  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    features = self.data.iloc[idx, :-1].values
    target = self.data.iloc[idx, -1]
    return features, target


In [ ]:
model = Lung_Cancer_Model()
model.to(device)

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
dataset = LungDataset(data)

batch_size = 100
dataset_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

num_epochs = 300

for epoch in range(num_epochs):
  for patient_data, diagnosis in dataset_loader:
    patient_data, diagnosis = patient_data.to(device), diagnosis.to(device)
    diagnosis = diagnosis.unsqueeze(-1)
    optimizer.zero_grad()
    outputs = model(patient_data)
    loss = criterion(outputs, diagnosis)
    loss.backward()
    optimizer.step()
  print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [1/300], Loss: 0.7706
Epoch [2/300], Loss: 0.6246
Epoch [3/300], Loss: 0.6827
Epoch [4/300], Loss: 0.6508
Epoch [5/300], Loss: 0.6943
Epoch [6/300], Loss: 0.7873
Epoch [7/300], Loss: 0.6601
Epoch [8/300], Loss: 0.7389
Epoch [9/300], Loss: 0.6453
Epoch [10/300], Loss: 0.6744
Epoch [11/300], Loss: 0.7375
Epoch [12/300], Loss: 0.7348
Epoch [13/300], Loss: 0.6082
Epoch [14/300], Loss: 0.6890
Epoch [15/300], Loss: 0.6985
Epoch [16/300], Loss: 0.7053
Epoch [17/300], Loss: 0.6553
Epoch [18/300], Loss: 0.6226
Epoch [19/300], Loss: 0.5592
Epoch [20/300], Loss: 0.5872
Epoch [21/300], Loss: 0.6329
Epoch [22/300], Loss: 0.6431
Epoch [23/300], Loss: 0.6437
Epoch [24/300], Loss: 0.6171
Epoch [25/300], Loss: 0.5993
Epoch [26/300], Loss: 0.5863
Epoch [27/300], Loss: 0.6553
Epoch [28/300], Loss: 0.5906
Epoch [29/300], Loss: 0.5703
Epoch [30/300], Loss: 0.5921
Epoch [31/300], Loss: 0.5667
Epoch [32/300], Loss: 0.5749
Epoch [33/300], Loss: 0.5865
Epoch [34/300], Loss: 0.5711
Epoch [35/300], Loss: 0

In [ ]:

def input_data(in_data):
  input_data = torch.tensor([np.array(in_data)])
  input_data = input_data.to(device)
  with torch.no_grad():
      return model(input_data)
predictions = input_data(data.iloc[87,:23].values)
predictions


tensor([[0.9951]], device='cuda:0')